## Script to add new extensions to FHIR Questionnaires

- python version 3.6+

- upload example as python dict from file
- add new extensions
- write to ig folder as json with text elements
- write to FHIR server as json without text elements

In [255]:
from json import load, dumps
from flatten_json import flatten
import pandas as pd
from csv import writer
import openpyxl
from IPython import display as D
from fhirclient.models import questionnaire as Q
from fhirclient.models import extension as X
from requests import get, post, put
from IPython.display import display, Markdown

### Open Questionnaires as DICT files

In [256]:
in_path = '/Users/ehaas/Documents/FHIR/Argo-Questionnaire/source/examples/'
out_path = '/Users/ehaas/Documents/FHIR/Argo-Questionnaire/source/examples/'



#### type in file name here

In [257]:
#in_file = ['ASQ3','AUDIT','DAST','Housing','PHQ9','Sampler']
in_file = ['Sampler']

In [258]:
q_parts = ['contained','item','text']

val_server = 'http://fhirtest.uhn.ca/baseDstu3'
fhir_test_server = 'http://sqlonfhir-stu3.azurewebsites.net/fhir'

headers = {
'Accept':'application/fhir+json',
'Content-Type':'application/fhir+json'
}

r_type='Questionnaire'

profile = 'http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/q'

In [259]:
def open_file(name): # get files
    with open(f'{in_path}{name}.json') as f:
        q = load(f)
        return(Q.Questionnaire(q))

### add timelimit extension

In [260]:
def get_timelimit_x(value,unit):
    time_unit_maps = dict(
     second = 's',
     minute = 'min',
     hour = 'h',
     day = 'd',
     week = 'wk',
     month = 'mo',
     year = 'a',
    )
    

    
    timelimit_x = X.Extension({
    'url':'http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-timelimit',
    'valueDuration': {
                        "value" : value,
                        "unit" : unit,
                        "system" : "http://unitsofmeasure.org",
                        "code" : time_unit_maps[unit] 
                        }
                    })

    return timelimit_x

a = get_timelimit_x(2,'year')
a.as_json()

{'url': 'http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-timelimit',
 'valueDuration': {'code': 'a',
  'system': 'http://unitsofmeasure.org',
  'unit': 'year',
  'value': 2}}


### Validate the Resource
Using the $validate operation, the example is validated by a FHIR Reference Server. The results are displayed below in the human readable text as xhtml.



In [261]:

# *********************** validate Resource ********************************
def validate(rjson):
    params = {
    'profile': profile
    }
    #   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
    r = post(f'{val_server}/{r_type}/$validate', params = params, headers = headers, data = rjson)
    # print(r.status_code)
    # view  output
    return r.json()["text"]["div"]
    

### Update Resource to FHIR Server

In [262]:

# *********************** update Resource on FHIR Server  ********************************
def update(id,data):
    
    params = {
    'profile': profile
    }
    #   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
    r = put(f'{fhir_test_server}/{r_type}/{id}', params = params, headers = headers, data = data)
    # print(r.status_code)
    # view  output
    return r.status_code
    

### Write to file

In [263]:
def write_file(path, name, data): # write file
    with open(f'{path}{name}.json', 'w') as f:
        f.write(data)


### Instantiate itemorder extension and assign value

In [264]:
def get_itemorder_x(i):
    itemorder_x = X.Extension({
    'url':'http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-itemOrder',
    'valueUnsignedInt': i
    })

    return itemorder_x



### add item order to each group and to each choice type questions


In [265]:
for name in in_file:
    # create fhirmodel
    q = open_file(name)
    
    print(q.id)
    print(q.extension)
    # add timelimit to questionnaire
    tl = 5
    tlu = 'minute'
    try:
        q.extension.append(get_timelimit_x(value=tl,unit=tlu))
    except AttributeError:
        q.extension = [get_timelimit_x(value=tl,unit=tlu)]
    for i in q.extension:
        print(dumps(i.as_json(),indent =3))
        
    for i,i_item in enumerate(q.item):
        print(f'i={i}, i_item={i_item.linkId}')
        # print(f'i_item.extension = {i_item.extension}')
        
        try:
            i_item.extension.append(get_itemorder_x(i))
        except AttributeError:
            i_item.extension = [get_itemorder_x(i)]
        for y in i_item.extension:
            pass # print(y.as_json())
        try:
            for j,j_item in enumerate(i_item.item):
                print(f'j={j}, j_item={j_item.linkId}')

                try:
                    j_item.extension.append(get_itemorder_x(j))
                except AttributeError:
                    j_item.extension = [get_itemorder_x(j)]
                for z in i_item.extension:
                    pass # print(z.as_json())
                try:
                    for jj, jopt in enumerate(j_item.option):

                        print(f'jj={jj}, iopt={jopt.as_json()}')

                        try:
                            jopt.extension.append(get_itemorder_x(jj))
                        except AttributeError:
                            jopt.extension = [get_itemorder_x(jj)]
                        print(f'jj={jj}, iopt={jopt.as_json()}')   
                except TypeError:
                    pass
        except TypeError:
            pass
            
    rjson = dumps(q.as_json(),indent=3)
    # write files as json
    write_file(path='', name=f'{name}-v2', data=rjson)
    # to IG examples file
    write_file(path =out_path, name=f'{name}-v2', data=rjson)
    # todo validate,  run in rendering-tool.

    #validate
    #remove the text attribute first
    q.text = None
    rjson = dumps(q.as_json())
    display(D.HTML(f'<h1>Validation output</h1>{validate(rjson)}'))
    
    #write to fhir server
    display(D.HTML(f'<h1>FHIR Server update output</h1>{update(id=q.id,data=rjson)}'))
    
    
    
    

questionnaire-example-sampler
None
{
   "url": "http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-timelimit",
   "valueDuration": {
      "code": "min",
      "system": "http://unitsofmeasure.org",
      "unit": "minute",
      "value": 5
   }
}
i=0, i_item=g1
j=0, j_item=g1.d
i=1, i_item=g2
j=0, j_item=g2.q1
i=2, i_item=g3
j=0, j_item=g3.q1
i=3, i_item=g4
j=0, j_item=g4.q1
i=4, i_item=g5
j=0, j_item=g5.q1
i=5, i_item=g6
j=0, j_item=g6.q1
i=6, i_item=g7
j=0, j_item=g7.q1
i=7, i_item=g8
j=0, j_item=g8.q1
i=8, i_item=g9
j=0, j_item=g9.q1
i=9, i_item=g10
j=0, j_item=g10.q1
i=10, i_item=g11
j=0, j_item=g11.q1
jj=0, iopt={'extension': [{'url': 'http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-score', 'valueDecimal': 0}], 'valueString': '10'}
jj=0, iopt={'extension': [{'url': 'http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-score', 'valueDecimal': 0}, {'url': 'http://fhir.org/guides/argonaut-questionnaire/Struct

WARNING,[Questionnaire.meta.profile[0]],"StructureDefinition reference ""http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/q"" could not be resolved"
INFORMATION,[Questionnaire.extension],Unknown extension http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-timelimit
WARNING,[Questionnaire.jurisdiction],"None of the codes provided are in the value set http://hl7.org/fhir/ValueSet/jurisdiction (http://hl7.org/fhir/ValueSet/jurisdiction, and a code should come from this value set unless it has no suitable code) (codes = urn:iso:std:iso:3166#US)"
INFORMATION,[Questionnaire.item[1].extension],Unknown extension http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-itemOrder
INFORMATION,[Questionnaire.item[1].item.extension],Unknown extension http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-itemOrder
WARNING,[Questionnaire.item[1].item.text],value should not start or finish with whitespace
INFORMATION,[Questionnaire.item[2].extension],Unknown extension http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-itemOrder
WARNING,[Questionnaire.item[2].prefix],value should not start or finish with whitespace
INFORMATION,[Questionnaire.item[2].item.extension[1]],Unknown extension http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-score
INFORMATION,[Questionnaire.item[2].item.extension[2]],Unknown extension http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-itemOrder
INFORMATION,[Questionnaire.item[3].extension],Unknown extension http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/extension-itemOrder


TODO:

- run in rendering-tool
- change to timelimit to timeLimit
- not part of QR - edit this out
- udpate comments change impossible to difficult